# Read the Docs Tutorial

This notebook is part of a tutorial called "[Build and Edit a MOM6 Grid in Jupyter](https://mom6gridtools.readthedocs.io/en/latest/tutorials/jupyterMOM6.html)".

In [ ]:
# Load some basic python libraries
import os, cartopy
# This loads the GridUtils portion of the gridtools library
from gridtools.gridutils import GridUtils

In [ ]:
# Create a GridUtils object
grd = GridUtils()

# Create a grid generation application object
grdGenApp = grd.app()

In [ ]:
# Launch the grid generation application within Jupyter
display(grdGenApp)

In [ ]:
# Obtain roughness and topography grids
runThisCodeBlock = False
if runThisCodeBlock:
    # Detach logger from application
    grd.detachLoggingFromApplication()
    
    # Source of GEBCO 2020 topographic grid
    highResTopographyFile = "/import/AKWATERS/jrcermakiii/bathy/gebco/GEBCO_2020.nc"

    if os.path.isfile(highResTopographyFile):
        topoGrids = grd.computeBathymetricRoughness(highResTopographyFile,
            depthName='elevation',
            maxMb=99, superGrid=False, useClipping=False,
            auxVariables=['depth'])    

In [ ]:
# Turn the diagnosed topography grid into an actual depth
runThisCodeBlock = False
if runThisCodeBlock:    
    topoGrids['depth'] = -(topoGrids['depth'])

In [ ]:
# Write FMS coupler and mosaic files, roughness and topography
runThisCodeBlock = False
if runThisCodeBlock:
    # Write current model grid files
    wrkDir = "/home/cermak/workdir/configs/zOutput"
    inputDir = os.path.join(wrkDir, "INPUT")
    input2Dir = os.path.join(wrkDir, "INPUT2")

    # Write FMS coupler and mosaic files
    grd.makeSoloMosaic(
        topographyGrid=topoGrids['depth'],
        writeLandmask=True,
        writeOceanmask=True,
        inputDirectory=inputDir,
        overwrite=True
    )

    # Write topographic variable
    topoGrids.to_netcdf(os.path.join(inputDir, 'ocean_topog.nc'),
            encoding=grd.removeFillValueAttributes(data=topoGrids))

    # Write the model grid
    grd.saveGrid(filename=os.path.join(inputDir, "ocean_hgrid.nc"))    

In [ ]:
# Write FMS coupler and mosaic files, roughness and topography
runThisCodeBlock = False
if runThisCodeBlock:
    # Examine the topography grid 
    grd.setPlotParameters({
        'figsize': (8,8),
        'projection': {
            'name': 'LambertConformalConic',
            'lon_0': 230.0,
            'lat_1': 25.0,
            'lat_2': 55.0
        },
        'extent': [-160.0 ,-100.0, 20.0, 60.0],
        'iLinewidth': 1.0,
        'jLinewidth': 1.0,
        'showGridCells': True,
        'iColor': 'k',
        'jColor': 'k',
        'transform': cartopy.crs.PlateCarree(),
        'satelliteHeight': 35785831.0
    })
    (figure, axes) = grd.plotGrid(showModelGrid = True,
            plotVariables={
            'depth': {
                'values': topoGrids['depth'],
                'title': 'Ocean topography (meters)',
                'cbar_kwargs': {
                    'orientation': 'horizontal',
                }
            }
        })
    display(figure)
    
    # Examine the ocean mask
    oceanMask = grd.openDataset(os.path.join(inputDir, 'ocean_mask.nc'))

    # Define our own color map (same used in mask editor)
    import matplotlib.pyplot as plt
    land_color = (0.6, 1.0, 0.6)
    sea_color  = (0.6, 0.6, 1.0)
    maskCM = plt.matplotlib.colors.ListedColormap(
        [land_color, sea_color], name='land/sea')    
    
    # MOM6 places lon and lat in x and y
    # x and y need to be lon and lat coordinates for the mask editor
    oceanMask = oceanMask.rename({
        'x': 'lon',
        'y': 'lat'
    })
    oceanMask = oceanMask.set_coords(['lon', 'lat'])
    
    (figureMask, axesMask) = grd.plotGrid(showModelGrid = True,
            plotVariables={
            'mask': {
                'values': oceanMask['mask'],
                'title': 'Ocean mask (1 = ocean)',
                'cmap': maskCM,
                'cbar_kwargs': {
                    'orientation': 'horizontal',
                }
            }
        })
    display(figureMask)
    
    # Zoom in to take a closer look
    grd.setPlotParameters({
        'extent': [-140.0 ,-120.0, 49.0, 59.0]
    })
    
    (figureMaskZoom, axesMaskZoom) = grd.plotGrid(showModelGrid = True,
            plotVariables={
            'mask': {
                'values': oceanMask['mask'],
                'title': 'Ocean mask (1 = ocean): Zoom',
                'cmap': maskCM,
                'cbar_kwargs': {
                    'orientation': 'horizontal',
                }
            }
        })
    display(figureMaskZoom)    
    

In [ ]:
# Use the editor to make some mask updates
runThisCodeBlock = False
if runThisCodeBlock:
    # Load the mask editor application module from gridtools
    from gridtools.app import maskEditor
    
    # Set a map projection for the mask editor to use
    crs = cartopy.crs.Orthographic(-140, 45)

    # Create the mask editor object
    appObj = maskEditor(crs=crs, ds=oceanMask['mask'])
    
    # Create the mask editor application object
    app = appObj.createMaskEditorApp()

    # Launch the application
    display(app)

In [ ]:
# Save the new ocean mask
runThisCodeBlock = False
if runThisCodeBlock:
    newMask = oceanMask['mask'].copy()
    newMask = newMask.reset_coords(names = ['lat', 'lon'])
    grd.saveDataset(os.path.join(inputDir, 'ocean_mask_new.nc'), newMask, 
                    overwrite=True, mapVariables = {'lon': 'x', 'lat': 'y'}, 
                    hashVariables = ['mask', 'x', 'y'])    

In [ ]:
# Apply new ocean mask to ocean model grid
runThisCodeBlock = False
if runThisCodeBlock:
    topoGrids['depth'] = grd.applyExistingOceanmask(topoGrids, 'depth',
        os.path.join(inputDir, 'ocean_mask_new.nc'), 'mask',
        MASKING_DEPTH=0.0, MINIMUM_DEPTH=0.0, MAXIMUM_DEPTH=-99999.0)

In [ ]:
# Rewrite FMS coupler and mosaic files
runThisCodeBlock = False
if runThisCodeBlock:
    grd.makeSoloMosaic(
        topographyGrid=topoGrids['depth'],
        writeLandmask=True,
        writeOceanmask=True,
        inputDirectory=input2Dir,
        overwrite=True,
        MASKING_DEPTH=0.0, MINIMUM_DEPTH=0.0, MAXIMUM_DEPTH=-99999.0
    )

    # Be sure to save previously diagnosed `h2` grid
    topoGrids.to_netcdf(os.path.join(input2Dir, 'ocean_topog.nc'),
            encoding=grd.removeFillValueAttributes(data=topoGrids))    
    
    grd.saveGrid(filename=os.path.join(input2Dir, "ocean_hgrid.nc"))

In [ ]:
# Replot the ocean mask to check
runThisCodeBlock = False
if runThisCodeBlock:
    (figureMaskZoom2, axesMaskZoom2) = grd.plotGrid(showModelGrid = True,
            plotVariables={
            'mask': {
                'values': oceanMask['mask'],
                'title': 'Ocean mask (1 = ocean): Zoom',
                'cmap': maskCM,
                'cbar_kwargs': {
                    'orientation': 'horizontal',
                }
            }
        })
    display(figureMaskZoom2)